In [5]:
from tkinter import *
import cv2 
from PIL import Image, ImageTk 
import numpy as np
import time
import math
import sys
from pyfirmata import Arduino

In [6]:
# Seting up pyfirmata params
board = Arduino('COM12')

# Left motor pins
enableLeftPin = board.get_pin('d:11:p')
input1LeftPin = board.get_pin('d:13:o')
input2LeftPin = board.get_pin('d:12:o')

# Right motor pins
enableRightPin = board.get_pin('d:10:p')
input3RightPin = board.get_pin('d:9:o')
input4RightPin = board.get_pin('d:8:o')


SerialException: [Errno 2] could not open port COM12: [Errno 2] No such file or directory: 'COM12'

In [10]:
# Setting Aruco params

# Pruned aruco type dict. See PoseEstimation.ipynb for full list
ARUCO_DICT = {"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL}

arucoDict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT["DICT_ARUCO_ORIGINAL"])

arucoParams = cv2.aruco.DetectorParameters()

# Camera Parameters
intrinsic_camera = np.array(((933.15867, 0, 657.59),(0,933.1586, 400.36993),(0,0,1)))
distortion = np.array((-0.43948,0.18514,0,0))

In [8]:
# defining  functions
def setMotorSpeeds(leftSpeed, rightSpeed):
        # Set left motor direction
        if leftSpeed >= 0:
            input1LeftPin.write(1)
            input2LeftPin.write(0)
        else:
            input1LeftPin.write(0)
            input2LeftPin.write(1)
            leftSpeed = -leftSpeed
      
        # Set right motor deirection
        if rightSpeed >= 0:
            input3RightPin.write(1)
            input4RightPin.write(0)
        else:
            input3RightPin.write(0)
            input4RightPin.write(1)
            rightSpeed = -rightSpeed
    
        enableLeftPin.write(leftSpeed/255)
        enableRightPin.write(rightSpeed/255)

def calcMotorSpeed_PID_TupleIn(currentPoseTuple, desiredPositionTuple, CCW_ANGLE_DIRECTION = 1, integralError = 0, derivativeError = 0):
    '''
    Parameters
    ----------
    robotPoseTuple : Tuple of floats
        (x, y, phi) Tuple describing the robot's current position and heading: .
    desiredPointTuple : Tuple of floats
        (x_desired, y_desired) Position of robot's current point it's tracking towards.
    CCW_ANGLE_DIRECTION: 1 or -1, default is 1
        Input -1 if rotating CCW decreases angle reading
    integralError : float, optional
        Integral Error, whatever way you like to calculate. The default is 0.
    derivativeError : float, optional
        Derivative Error, whatever way you like to calculate. The default is 0.

    Returns
    -------
    motorSpeedTuple : Tuple of Floats
        (leftSpeed, rightSpeed) speeds as float vary from -255 to 255.
    '''
    # Process Input
    x = currentPoseTuple[0];
    y = currentPoseTuple[1];
    phi = currentPoseTuple[2];
    x_desired = desiredPositionTuple[0];
    y_desired = desiredPositionTuple[1];
    # Constant Parameters: 
    kp = 8 #proportional constant
    ki = 0 #integralError constant
    kd = 0 #derivativeError constant
    ANGLE_OF_GOING_RIGHT = -90; #angle (deg) the robot detects when moving right. Set to -90 if up is 0, CCW is positive.
    DEFAULT_SPEED = 255 #the regular speed the robot runs at
    MAX_SPEED = 255 #exactly 255
    MIN_SPEED = -255;
    CCW_ANGLE_DIRECTION = 1; #set to 1 if CCW is negative
    ######

    # Same code as before
    dx = x_desired - x;
    dy = y_desired - y;
    leftSpeed = DEFAULT_SPEED
    rightSpeed = DEFAULT_SPEED
    angleToDesired = math.degrees(math.atan2(dy, dx)); #
    currentAngle = phi - ANGLE_OF_GOING_RIGHT
    
    #Positive error means need to move CCW
    error = angleToDesired - currentAngle;
    error = CCW_ANGLE_DIRECTION * error;
    
    u = kp * error + ki * integralError + kd * derivativeError
    
    leftSpeed = max(MIN_SPEED, min(MAX_SPEED, leftSpeed - u)) 
    rightSpeed = max(MIN_SPEED, min(MAX_SPEED, rightSpeed + u))
    motorSpeedTuple = (leftSpeed, rightSpeed)
    return motorSpeedTuple #return a tuple, of the new motor speeds

def generate_lawnmower_path(xTopLeft, yTopLeft, xBottomRight, yBottomRight, L, ds=2):
    '''
    Parameters
    ----------
    xTopLeft : float
        X coordinate of Top Left (or bottom left) point of bounding box.
    yTopLeft : float
        Y coordinate of Top Left (or bottom left) point of bounding box.
    xBottomRight : float
        X coordinate of Bottom Right (or top right) point of bounding box.
    yBottomRight : float
        X coordinate of Bottom Right (or top right) point of bounding box.
    L : float
        Distance between zig zags. Input the width of the robot or something.
    ds : float, optional
        The distance between points. The default is 2 units.

    Returns
    -------
    points : List of tuples, where each row/tuple is (x,y) of that point along path
        DESCRIPTION.

    '''
    points = []
    x_current = xTopLeft
    y_current = yTopLeft

    width = xBottomRight - xTopLeft
    height = yBottomRight - yTopLeft

    # Ensure that height is positive, assuming the direction is downward
    if height < 0:
        height = -height
        y_direction = -1
    else:
        y_direction = 1

    direction = 1  # 1 for right, -1 for left

    while (y_direction == 1 and y_current < yBottomRight) or (y_direction == -1 and y_current > yBottomRight):
        # Horizontal movement
        if direction == 1:
            while x_current < xBottomRight:
                points.append((x_current, y_current))
                x_current += ds
                if x_current >= xBottomRight:
                    x_current = xBottomRight
                    points.append((x_current, y_current))
                    break
        else:
            while x_current > xTopLeft:
                points.append((x_current, y_current))
                x_current -= ds
                if x_current <= xTopLeft:
                    x_current = xTopLeft
                    points.append((x_current, y_current))
                    break

        # Vertical movement
        y_next = y_current + y_direction * L
        if y_direction == 1 and y_next >= yBottomRight:
            y_next = yBottomRight
        elif y_direction == -1 and y_next <= yBottomRight:
            y_next = yBottomRight
        while (y_direction == 1 and y_current < y_next) or (y_direction == -1 and y_current > y_next):
            y_current += y_direction * ds
            if (y_direction == 1 and y_current > y_next) or (y_direction == -1 and y_current < y_next):
                y_current = y_next
            points.append((x_current, y_current))

        direction *= -1

    return points


In [9]:
# Function for capturing video frame and displaying it.
# Schedules itself to run again in 10ms
def open_camera():
    global initiate_cleaning, cleaning, pts, i, x1, y1, x2, y2, xhome, yhome
    # Capture the video frame by frame 
    _, frame = vid.read()

    # Initialize variables with default values
    x1_int = x1
    y1_int = y1
    x2_int = x2
    y2_int = y2
    xhome_int = xhome
    yhome_int = yhome

    # Draw the bounding box
    try:
        x1_int = int(x1_var.get())
        y1_int = int(y1_var.get())
        x2_int = int(x2_var.get())
        y2_int = int(y2_var.get())
    except ValueError:
        pass  # Use default values if the inputs are not valid integers

    cv2.rectangle(frame, (x1_int, y1_int), (x2_int, y2_int), (255, 0, 0), 2)
    
    # Draw the home position
    try:
        xhome_int = int(xhome_var.get())
        yhome_int = int(yhome_var.get())
        cv2.circle(frame, (xhome_int, yhome_int), 5, (0, 255, 0), -1)
    except ValueError:
        pass  # Handle the case where the inputs are not valid integers

    # Convert image from one color space to other 
    opencv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    
    # Capture the latest frame and transform to image 
    captured_image = Image.fromarray(opencv_image)

    # Convert captured image to photoimage 
    photo_image = ImageTk.PhotoImage(image=captured_image)

    # Displaying photoimage in the label 
    label_widget.photo_image = photo_image

    # Configure image in the label 
    label_widget.configure(image=photo_image)

    robotWidth = 20
    # Run the following if the "Clean" button has been clicked
    if initiate_cleaning and not cleaning:
        pts = generate_lawnmower_path(x1_int, y1_int, x2_int, y2_int, 0.75 * robotWidth, ds=2)
        i = 0
        initiate_cleaning = False
        cleaning = True

    if cleaning:
        # if dist to pts(i) < threshold
        # if i == length(pts): cleaning = False, motor speeds = 0
        # else: i++, reset error vals

        robotPos = (0, 0, 0)
        pt = (1, 1)

        # run PID
        # (leftSpeed, rightSpeed) = calcMotorSpeed_PID(robotPos, pt, 0, 0)
        (leftSpeed, rightSpeed) = calcMotorSpeed_PID_TupleIn(robotPos, pt)
        # print(leftSpeed)
        # print(rightSpeed)
        setMotorSpeeds(leftSpeed, rightSpeed)

    # Repeat the same process after every 10 milliseconds 
    label_widget.after(10, open_camera)

# Function to handle mouse click events
def on_mouse_click(event):
    global x1, y1, x2, y2, xhome, yhome, bounding_box_selected, home_selected, click_count

    if bounding_box_selected:
        if click_count == 0:
            x1, y1 = event.x, event.y
            x1_var.set(x1)
            y1_var.set(y1)
            click_count += 1
        elif click_count == 1:
            x2, y2 = event.x, event.y
            x2_var.set(x2)
            y2_var.set(y2)
            click_count = 0
            bounding_box_selected = False
    elif home_selected:
        xhome, yhome = event.x, event.y
        xhome_var.set(xhome)
        yhome_var.set(yhome)
        click_count = 0
        home_selected = False

# Function to start bounding box selection
def choose_bounding_box():
    global bounding_box_selected, home_selected, click_count
    bounding_box_selected = True
    home_selected = False
    click_count = 0

# Function to start home selection
def choose_home():
    global home_selected, bounding_box_selected
    home_selected = True
    bounding_box_selected = False
    click_count = 0

# Function to set cleaning initiation
def start_cleaning():
    global initiate_cleaning
    initiate_cleaning = True

# Function to stop the robot
def stop_robot():
    global cleaning
    setMotorSpeeds(0, 0)
    cleaning = False

# Define a video capture object 
vid = cv2.VideoCapture(0) 

# Declare the width and height in variables 
width, height = 1920, 1080

# Set the width and height 
# adjust to match phone resolution
vid.set(cv2.CAP_PROP_FRAME_WIDTH, width) 
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, height) 

# Create a GUI app 
app = Tk() 
app.state('zoomed')

# Bind the app with Escape keyboard to 
# quit app whenever pressed 
# app.bind('<Escape>', lambda e: app.quit())

# Function to release the camera when the window is closed
def on_closing():
    vid.release()
    app.destroy()
    # board.sp.close()

app.protocol("WM_DELETE_WINDOW", on_closing)

# Create a label and display it on app 
label_widget = Label(app) 
label_widget.pack() 
label_widget.bind('<Button-1>', on_mouse_click)

# Create a frame to hold all controls
control_frame = Frame(app)
control_frame.pack()

# Open Camera button on the left
open_camera_button = Button(control_frame, text="Open Camera", command=open_camera)
open_camera_button.grid(row=0, column=0, padx=10, pady=10)

# Create input fields and labels for user-defined cleaning boundaries
x1_var = StringVar()
y1_var = StringVar()
x2_var = StringVar()
y2_var = StringVar()
xhome_var = StringVar()
yhome_var = StringVar()

# Home coordinates and button
home_frame = Frame(control_frame)
home_frame.grid(row=0, column=1, padx=10, pady=10)
home_button = Button(home_frame, text="Choose Home", command=choose_home)
home_button.grid(row=0, column=0, rowspan=2, padx=10)

label_xhome = Label(home_frame, text="x home:")
label_xhome.grid(row=0, column=1)
entry_xhome = Entry(home_frame, textvariable=xhome_var)
entry_xhome.grid(row=0, column=2)

label_yhome = Label(home_frame, text="y home:")
label_yhome.grid(row=1, column=1)
entry_yhome = Entry(home_frame, textvariable=yhome_var)
entry_yhome.grid(row=1, column=2)

# Bounding box coordinates and button
bbox_frame = Frame(control_frame)
bbox_frame.grid(row=0, column=2, padx=10, pady=10)
bounding_box_button = Button(bbox_frame, text="Choose Bounding Box", command=choose_bounding_box)
bounding_box_button.grid(row=0, column=0, rowspan=2, padx=10)

label_x1 = Label(bbox_frame, text="x1:")
label_x1.grid(row=0, column=1)
entry_x1 = Entry(bbox_frame, textvariable=x1_var)
entry_x1.grid(row=0, column=2)

label_y1 = Label(bbox_frame, text="y1:")
label_y1.grid(row=1, column=1)
entry_y1 = Entry(bbox_frame, textvariable=y1_var)
entry_y1.grid(row=1, column=2)

label_x2 = Label(bbox_frame, text="x2:")
label_x2.grid(row=0, column=3)
entry_x2 = Entry(bbox_frame, textvariable=x2_var)
entry_x2.grid(row=0, column=4)

label_y2 = Label(bbox_frame, text="y2:")
label_y2.grid(row=1, column=3)
entry_y2 = Entry(bbox_frame, textvariable=y2_var)
entry_y2.grid(row=1, column=4)

# Start Cleaning button on the right
start_button = Button(control_frame, text="Start Cleaning", command=start_cleaning)
start_button.grid(row=0, column=3, padx=10, pady=10)

# Stop Robot button on the right
stop_button = Button(control_frame, text="Stop Robot", command=stop_robot)
stop_button.grid(row=0, column=4, padx=10, pady=10)

# Initialize state as not cleaning
cleaning = False  # Usually False
initiate_cleaning = False

# Initialize array of desired pts and all control variables
pts = []
i = 0

# Variables for bounding box and home selection
bounding_box_selected = False
home_selected = False
click_count = 0
x1, y1, x2, y2 = 0, 0, 0, 0
xhome, yhome = 0, 0

# Create an infinite loop for displaying app on screen
app.mainloop()
